<a href="https://colab.research.google.com/github/sbooeshaghi/azucar/blob/main/analysis/293T/obs2/assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title sklearn/mixture/_base.py

"""Base class for mixture models."""

# sklearn/mixture/_base.py

# Author: Wei Xue <xuewei4d@gmail.com>
# Modified by Thierry Guillemot <thierry.guillemot.work@gmail.com>
# License: BSD 3 clause

import warnings
from abc import ABCMeta, abstractmethod
from time import time

import numpy as np
from scipy.special import logsumexp

from sklearn import cluster
from sklearn.base import BaseEstimator
from sklearn.base import DensityMixin
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils import check_random_state
from sklearn.utils.validation import check_is_fitted


def _check_shape(param, param_shape, name):
    """Validate the shape of the input parameter 'param'.

    Parameters
    ----------
    param : array

    param_shape : tuple

    name : str
    """
    param = np.array(param)
    if param.shape != param_shape:
        raise ValueError(
            "The parameter '%s' should have the shape of %s, but got %s" %
            (name, param_shape, param.shape))


class BaseMixture(DensityMixin, BaseEstimator, metaclass=ABCMeta):
    """Base class for mixture models.

    This abstract class specifies an interface for all mixture classes and
    provides basic common methods for mixture models.
    """
    def __init__(
        self,
        n_components,
        tol,
        reg_covar,
        max_iter,
        n_init,
        init_params,
        random_state,
        warm_start,
        verbose,
        verbose_interval,
    ):
        self.n_components = n_components
        self.tol = tol
        self.reg_covar = reg_covar
        self.max_iter = max_iter
        self.n_init = n_init
        self.init_params = init_params
        self.random_state = random_state
        self.warm_start = warm_start
        self.verbose = verbose
        self.verbose_interval = verbose_interval

    def _check_initial_parameters(self, X):
        """Check values of the basic parameters.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        """
        if self.n_components < 1:
            raise ValueError("Invalid value for 'n_components': %d "
                             "Estimation requires at least one component" %
                             self.n_components)

        if self.tol < 0.0:
            raise ValueError("Invalid value for 'tol': %.5f "
                             "Tolerance used by the EM must be non-negative" %
                             self.tol)

        if self.n_init < 1:
            raise ValueError(
                "Invalid value for 'n_init': %d Estimation requires at least one run"
                % self.n_init)

        if self.max_iter < 1:
            raise ValueError("Invalid value for 'max_iter': %d "
                             "Estimation requires at least one iteration" %
                             self.max_iter)

        if self.reg_covar < 0.0:
            raise ValueError("Invalid value for 'reg_covar': %.5f "
                             "regularization on covariance must be "
                             "non-negative" % self.reg_covar)

        # Check all the parameters values of the derived class
        self._check_parameters(X)

    @abstractmethod
    def _check_parameters(self, X):
        """Check initial parameters of the derived class.

        Parameters
        ----------
        X : array-like of shape  (n_samples, n_features)
        """
        pass

    def _initialize_parameters(self, X, random_state, B=None, resp=None):
        """Initialize the model parameters.

        Parameters
        ----------
        X : array-like of shape  (n_samples, n_features)

        random_state : RandomState
            A random number generator instance that controls the random seed
            used for the method chosen to initialize the parameters.
        """
        n_samples, _ = X.shape

        if self.init_params == "kmeans":
            resp = np.zeros((n_samples, self.n_components))
            label = (cluster.KMeans(n_clusters=self.n_components,
                                    n_init=1,
                                    random_state=random_state).fit(X).labels_)
            resp[np.arange(n_samples), label] = 1
        elif self.init_params == "random":
          if resp is None:
            resp = random_state.rand(n_samples, self.n_components)
            resp /= resp.sum(axis=1)[:, np.newaxis]
        else:
            raise ValueError("Unimplemented initialization method '%s'" %
                             self.init_params)

        self._initialize(X, resp, B)

    @abstractmethod
    def _initialize(self, X, resp, B=None):
        """Initialize the model parameters of the derived class.

        Parameters
        ----------
        X : array-like of shape  (n_samples, n_features)

        resp : array-like of shape (n_samples, n_components)
        """
        pass

    def fit(self, X, y=None):
        """Estimate model parameters with the EM algorithm.

        The method fits the model ``n_init`` times and sets the parameters with
        which the model has the largest likelihood or lower bound. Within each
        trial, the method iterates between E-step and M-step for ``max_iter``
        times until the change of likelihood or lower bound is less than
        ``tol``, otherwise, a ``ConvergenceWarning`` is raised.
        If ``warm_start`` is ``True``, then ``n_init`` is ignored and a single
        initialization is performed upon the first call. Upon consecutive
        calls, training starts where it left off.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.

        y : Ignored
            Not used, present for API consistency by convention.

        Returns
        -------
        self : object
            The fitted mixture.
        """
        self.fit_predict(X, y)
        return self

    def fit_predict(self, X, y=None, B=None, resp=None):
        """Estimate model parameters using X and predict the labels for X.

        The method fits the model n_init times and sets the parameters with
        which the model has the largest likelihood or lower bound. Within each
        trial, the method iterates between E-step and M-step for `max_iter`
        times until the change of likelihood or lower bound is less than
        `tol`, otherwise, a :class:`~sklearn.exceptions.ConvergenceWarning` is
        raised. After fitting, it predicts the most probable label for the
        input data points.

        .. versionadded:: 0.20

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.

        y : Ignored
            Not used, present for API consistency by convention.

        Returns
        -------
        labels : array, shape (n_samples,)
            Component labels.
        """
        X = self._validate_data(X,
                                dtype=[np.float64, np.float32],
                                ensure_min_samples=2)
        if X.shape[0] < self.n_components:
            raise ValueError("Expected n_samples >= n_components "
                             f"but got n_components = {self.n_components}, "
                             f"n_samples = {X.shape[0]}")
        self._check_initial_parameters(X)

        # if we enable warm_start, we will have a unique initialisation
        do_init = not (self.warm_start and hasattr(self, "converged_"))
        n_init = self.n_init if do_init else 1

        max_lower_bound = -np.inf
        self.converged_ = False

        random_state = check_random_state(self.random_state)

        n_samples, _ = X.shape
        for init in range(n_init):
            self._print_verbose_msg_init_beg(init)

            if do_init:
                self._initialize_parameters(X, random_state, B=B, resp=resp)

            lower_bound = -np.inf if do_init else self.lower_bound_

            for n_iter in range(1, self.max_iter + 1):
                prev_lower_bound = lower_bound

                log_prob_norm, log_resp = self._e_step(X)
                self._m_step(X, log_resp, B)
                lower_bound = self._compute_lower_bound(
                    log_resp, log_prob_norm)

                change = lower_bound - prev_lower_bound
                self._print_verbose_msg_iter_end(n_iter, change)

                if abs(change) < self.tol:
                    self.converged_ = True
                    break

            self._print_verbose_msg_init_end(lower_bound)

            if lower_bound > max_lower_bound or max_lower_bound == -np.inf:
                max_lower_bound = lower_bound
                best_params = self._get_parameters()
                best_n_iter = n_iter

        if not self.converged_:
            warnings.warn(
                "Initialization %d did not converge. "
                "Try different init parameters, "
                "or increase max_iter, tol "
                "or check for degenerate data." % (init + 1),
                ConvergenceWarning,
            )

        self._set_parameters(best_params)
        self.n_iter_ = best_n_iter
        self.lower_bound_ = max_lower_bound

        # Always do a final e-step to guarantee that the labels returned by
        # fit_predict(X) are always consistent with fit(X).predict(X)
        # for any value of max_iter and tol (and any random_state).
        _, log_resp = self._e_step(X)

        return log_resp.argmax(axis=1)

    def _e_step(self, X):
        """E step.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        log_prob_norm : float
            Mean of the logarithms of the probabilities of each sample in X

        log_responsibility : array, shape (n_samples, n_components)
            Logarithm of the posterior probabilities (or responsibilities) of
            the point of each sample in X.
        """
        log_prob_norm, log_resp = self._estimate_log_prob_resp(X)
        return np.mean(log_prob_norm), log_resp

    @abstractmethod
    def _m_step(self, X, log_resp, B=None):
        """M step.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        log_resp : array-like of shape (n_samples, n_components)
            Logarithm of the posterior probabilities (or responsibilities) of
            the point of each sample in X.
        """
        pass

    @abstractmethod
    def _get_parameters(self):
        pass

    @abstractmethod
    def _set_parameters(self, params):
        pass

    def score_samples(self, X):
        """Compute the log-likelihood of each sample.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.

        Returns
        -------
        log_prob : array, shape (n_samples,)
            Log-likelihood of each sample in `X` under the current model.
        """
        check_is_fitted(self)
        X = self._validate_data(X, reset=False)

        return logsumexp(self._estimate_weighted_log_prob(X), axis=1)

    def score(self, X, y=None):
        """Compute the per-sample average log-likelihood of the given data X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_dimensions)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.

        y : Ignored
            Not used, present for API consistency by convention.

        Returns
        -------
        log_likelihood : float
            Log-likelihood of `X` under the Gaussian mixture model.
        """
        return self.score_samples(X).mean()

    def predict(self, X):
        """Predict the labels for the data samples in X using trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.

        Returns
        -------
        labels : array, shape (n_samples,)
            Component labels.
        """
        check_is_fitted(self)
        X = self._validate_data(X, reset=False)
        return self._estimate_weighted_log_prob(X).argmax(axis=1)

    def predict_proba(self, X):
        """Evaluate the components' density for each sample.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            List of n_features-dimensional data points. Each row
            corresponds to a single data point.

        Returns
        -------
        resp : array, shape (n_samples, n_components)
            Density of each Gaussian component for each sample in X.
        """
        check_is_fitted(self)
        X = self._validate_data(X, reset=False)
        _, log_resp = self._estimate_log_prob_resp(X)
        return np.exp(log_resp)

    def sample(self, n_samples=1):
        """Generate random samples from the fitted Gaussian distribution.

        Parameters
        ----------
        n_samples : int, default=1
            Number of samples to generate.

        Returns
        -------
        X : array, shape (n_samples, n_features)
            Randomly generated sample.

        y : array, shape (nsamples,)
            Component labels.
        """
        check_is_fitted(self)

        if n_samples < 1:
            raise ValueError(
                "Invalid value for 'n_samples': %d . The sampling requires at "
                "least one sample." % (self.n_components))

        _, n_features = self.means_.shape
        rng = check_random_state(self.random_state)
        n_samples_comp = rng.multinomial(n_samples, self.weights_)

        if self.covariance_type == "full":
            X = np.vstack([
                rng.multivariate_normal(mean, covariance, int(sample))
                for (mean, covariance, sample
                     ) in zip(self.means_, self.covariances_, n_samples_comp)
            ])
        elif self.covariance_type == "tied":
            X = np.vstack([
                rng.multivariate_normal(mean, self.covariances_, int(sample))
                for (mean, sample) in zip(self.means_, n_samples_comp)
            ])
        else:
            X = np.vstack([
                mean + rng.randn(sample, n_features) * np.sqrt(covariance)
                for (mean, covariance, sample
                     ) in zip(self.means_, self.covariances_, n_samples_comp)
            ])

        y = np.concatenate([
            np.full(sample, j, dtype=int)
            for j, sample in enumerate(n_samples_comp)
        ])

        return (X, y)

    def _estimate_weighted_log_prob(self, X):
        """Estimate the weighted log-probabilities, log P(X | Z) + log weights.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        weighted_log_prob : array, shape (n_samples, n_component)
        """
        return self._estimate_log_prob(X) + self._estimate_log_weights()

    @abstractmethod
    def _estimate_log_weights(self):
        """Estimate log-weights in EM algorithm, E[ log pi ] in VB algorithm.

        Returns
        -------
        log_weight : array, shape (n_components, )
        """
        pass

    @abstractmethod
    def _estimate_log_prob(self, X):
        """Estimate the log-probabilities log P(X | Z).

        Compute the log-probabilities per each component for each sample.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        log_prob : array, shape (n_samples, n_component)
        """
        pass

    def _estimate_log_prob_resp(self, X):
        """Estimate log probabilities and responsibilities for each sample.

        Compute the log probabilities, weighted log probabilities per
        component and responsibilities for each sample in X with respect to
        the current state of the model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        log_prob_norm : array, shape (n_samples,)
            log p(X)

        log_responsibilities : array, shape (n_samples, n_components)
            logarithm of the responsibilities
        """
        weighted_log_prob = self._estimate_weighted_log_prob(X)
        log_prob_norm = logsumexp(weighted_log_prob, axis=1)
        with np.errstate(under="ignore"):
            # ignore underflow
            log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
        return log_prob_norm, log_resp

    def _print_verbose_msg_init_beg(self, n_init):
        """Print verbose message on initialization."""
        if self.verbose == 1:
            print("Initialization %d" % n_init)
        elif self.verbose >= 2:
            print("Initialization %d" % n_init)
            self._init_prev_time = time()
            self._iter_prev_time = self._init_prev_time

    def _print_verbose_msg_iter_end(self, n_iter, diff_ll):
        """Print verbose message on initialization."""
        if n_iter % self.verbose_interval == 0:
            if self.verbose == 1:
                print("  Iteration %d" % n_iter)
            elif self.verbose >= 2:
                cur_time = time()
                print("  Iteration %d\t time lapse %.5fs\t ll change %.5f" %
                      (n_iter, cur_time - self._iter_prev_time, diff_ll))
                self._iter_prev_time = cur_time

    def _print_verbose_msg_init_end(self, ll):
        """Print verbose message on the end of iteration."""
        if self.verbose == 1:
            print("Initialization converged: %s" % self.converged_)
        elif self.verbose >= 2:
            print("Initialization converged: %s\t time lapse %.5fs\t ll %.5f" %
                  (self.converged_, time() - self._init_prev_time, ll))

In [2]:
#@title sklearn/mixture/_gaussian_mixture.py 

"""Gaussian Mixture Model."""

# sklearn/mixture/_gaussian_mixture.py 

# Author: Wei Xue <xuewei4d@gmail.com>
# Modified by Thierry Guillemot <thierry.guillemot.work@gmail.com>
# License: BSD 3 clause

import numpy as np

from scipy import linalg

# from ._base import BaseMixture, _check_shape these come from cell above
from sklearn.utils import check_array
from sklearn.utils.extmath import row_norms

###############################################################################
# Gaussian mixture shape checkers used by the GaussianMixture class


def _check_weights(weights, n_components):
    """Check the user provided 'weights'.

    Parameters
    ----------
    weights : array-like of shape (n_components,)
        The proportions of components of each mixture.

    n_components : int
        Number of components.

    Returns
    -------
    weights : array, shape (n_components,)
    """
    weights = check_array(weights,
                          dtype=[np.float64, np.float32],
                          ensure_2d=False)
    _check_shape(weights, (n_components, ), "weights")

    # check range
    if any(np.less(weights, 0.0)) or any(np.greater(weights, 1.0)):
        raise ValueError("The parameter 'weights' should be in the range "
                         "[0, 1], but got max value %.5f, min value %.5f" %
                         (np.min(weights), np.max(weights)))

    # check normalization
    if not np.allclose(np.abs(1.0 - np.sum(weights)), 0.0):
        raise ValueError(
            "The parameter 'weights' should be normalized, but got sum(weights) = %.5f"
            % np.sum(weights))
    return weights


def _check_means(means, n_components, n_features):
    """Validate the provided 'means'.

    Parameters
    ----------
    means : array-like of shape (n_components, n_features)
        The centers of the current components.

    n_components : int
        Number of components.

    n_features : int
        Number of features.

    Returns
    -------
    means : array, (n_components, n_features)
    """
    means = check_array(means, dtype=[np.float64, np.float32], ensure_2d=False)
    _check_shape(means, (n_components, n_features), "means")
    return means


def _check_precision_positivity(precision, covariance_type):
    """Check a precision vector is positive-definite."""
    if np.any(np.less_equal(precision, 0.0)):
        raise ValueError("'%s precision' should be positive" % covariance_type)


def _check_precision_matrix(precision, covariance_type):
    """Check a precision matrix is symmetric and positive-definite."""
    if not (np.allclose(precision, precision.T)
            and np.all(linalg.eigvalsh(precision) > 0.0)):
        raise ValueError(
            "'%s precision' should be symmetric, positive-definite" %
            covariance_type)


def _check_precisions_full(precisions, covariance_type):
    """Check the precision matrices are symmetric and positive-definite."""
    for prec in precisions:
        _check_precision_matrix(prec, covariance_type)


def _check_precisions(precisions, covariance_type, n_components, n_features):
    """Validate user provided precisions.

    Parameters
    ----------
    precisions : array-like
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)

    covariance_type : str

    n_components : int
        Number of components.

    n_features : int
        Number of features.

    Returns
    -------
    precisions : array
    """
    precisions = check_array(
        precisions,
        dtype=[np.float64, np.float32],
        ensure_2d=False,
        allow_nd=covariance_type == "full",
    )

    precisions_shape = {
        "full": (n_components, n_features, n_features),
        "tied": (n_features, n_features),
        "diag": (n_components, n_features),
        "spherical": (n_components, ),
    }
    _check_shape(precisions, precisions_shape[covariance_type],
                 "%s precision" % covariance_type)

    _check_precisions = {
        "full": _check_precisions_full,
        "tied": _check_precision_matrix,
        "diag": _check_precision_positivity,
        "spherical": _check_precision_positivity,
    }
    _check_precisions[covariance_type](precisions, covariance_type)
    return precisions


###############################################################################
# Gaussian mixture parameters estimators (used by the M-Step)


def _estimate_gaussian_covariances_full(resp, X, nk, means, reg_covar):
    """Estimate the full covariance matrices.

    Parameters
    ----------
    resp : array-like of shape (n_samples, n_components)

    X : array-like of shape (n_samples, n_features)

    nk : array-like of shape (n_components,)

    means : array-like of shape (n_components, n_features)

    reg_covar : float

    Returns
    -------
    covariances : array, shape (n_components, n_features, n_features)
        The covariance matrix of the current components.
    """
    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[::n_features + 1] += reg_covar
    return covariances


def _estimate_gaussian_covariances_tied(resp, X, nk, means, reg_covar):
    """Estimate the tied covariance matrix.

    Parameters
    ----------
    resp : array-like of shape (n_samples, n_components)

    X : array-like of shape (n_samples, n_features)

    nk : array-like of shape (n_components,)

    means : array-like of shape (n_components, n_features)

    reg_covar : float

    Returns
    -------
    covariance : array, shape (n_features, n_features)
        The tied covariance matrix of the components.
    """
    avg_X2 = np.dot(X.T, X)
    avg_means2 = np.dot(nk * means.T, means)
    covariance = avg_X2 - avg_means2
    covariance /= nk.sum()
    covariance.flat[::len(covariance) + 1] += reg_covar
    return covariance


def _estimate_gaussian_covariances_diag(resp, X, nk, means, reg_covar):
    """Estimate the diagonal covariance vectors.

    Parameters
    ----------
    responsibilities : array-like of shape (n_samples, n_components)

    X : array-like of shape (n_samples, n_features)

    nk : array-like of shape (n_components,)

    means : array-like of shape (n_components, n_features)

    reg_covar : float

    Returns
    -------
    covariances : array, shape (n_components, n_features)
        The covariance vector of the current components.
    """
    avg_X2 = np.dot(resp.T, X * X) / nk[:, np.newaxis]
    avg_means2 = means**2
    avg_X_means = means * np.dot(resp.T, X) / nk[:, np.newaxis]
    return avg_X2 - 2 * avg_X_means + avg_means2 + reg_covar


def _estimate_gaussian_covariances_spherical(resp, X, nk, means, reg_covar):
    """Estimate the spherical variance values.

    Parameters
    ----------
    responsibilities : array-like of shape (n_samples, n_components)

    X : array-like of shape (n_samples, n_features)

    nk : array-like of shape (n_components,)

    means : array-like of shape (n_components, n_features)

    reg_covar : float

    Returns
    -------
    variances : array, shape (n_components,)
        The variance values of each components.
    """
    return _estimate_gaussian_covariances_diag(resp, X, nk, means,
                                               reg_covar).mean(1)


def _estimate_gaussian_parameters(X, resp, reg_covar, covariance_type, B=None):
    """Estimate the Gaussian distribution parameters.

    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        The input data array.

    resp : array-like of shape (n_samples, n_components)
        The responsibilities for each data sample in X.

    reg_covar : float
        The regularization added to the diagonal of the covariance matrices.

    covariance_type : {'full', 'tied', 'diag', 'spherical'}
        The type of precision matrices.

    Returns
    -------
    nk : array-like of shape (n_components,)
        The numbers of data samples in the current components.

    means : array-like of shape (n_components, n_features)
        The centers of the current components.

    covariances : array-like
        The covariance matrix of the current components.
        The shape depends of the covariance_type.
    """
    # print("Doing the thing..")
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]

    # frankie
    # get the mins for the marker genes
    # ct_mins = [means[:, i].min() for i in B]
    marker_gene_indices = [set(np.where(i)[0]) for i in B]
    ct_mins = [means[i][B[i]].min() for i in range(means.shape[0])]
    marker_gene_indices = [set(B[i]) for i in range(means.shape[0])]
    # modify based on the min/f
    f = 2.
    for idx, i in enumerate(means):
        ct_min = ct_mins[idx]
        betas = means[idx]
        for jdx, b in enumerate(betas):
            if jdx not in marker_gene_indices[idx]:
                new = 1e-5 # min(b, ct_min / f)
                means[idx][jdx] = new

    covariances = {
        "full": _estimate_gaussian_covariances_full,
        "tied": _estimate_gaussian_covariances_tied,
        "diag": _estimate_gaussian_covariances_diag,
        "spherical": _estimate_gaussian_covariances_spherical,
    }[covariance_type](resp, X, nk, means, reg_covar)
    return nk, means, covariances


def _compute_precision_cholesky(covariances, covariance_type):
    """Compute the Cholesky decomposition of the precisions.

    Parameters
    ----------
    covariances : array-like
        The covariance matrix of the current components.
        The shape depends of the covariance_type.

    covariance_type : {'full', 'tied', 'diag', 'spherical'}
        The type of precision matrices.

    Returns
    -------
    precisions_cholesky : array-like
        The cholesky decomposition of sample precisions of the current
        components. The shape depends of the covariance_type.
    """
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar.")

    if covariance_type == "full":
        n_components, n_features, _ = covariances.shape
        precisions_chol = np.empty((n_components, n_features, n_features))
        for k, covariance in enumerate(covariances):
            try:
                cov_chol = linalg.cholesky(covariance, lower=True)
            except linalg.LinAlgError:
                raise ValueError(estimate_precision_error_message)
            precisions_chol[k] = linalg.solve_triangular(cov_chol,
                                                         np.eye(n_features),
                                                         lower=True).T
    elif covariance_type == "tied":
        _, n_features = covariances.shape
        try:
            cov_chol = linalg.cholesky(covariances, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol = linalg.solve_triangular(cov_chol,
                                                  np.eye(n_features),
                                                  lower=True).T
    else:
        if np.any(np.less_equal(covariances, 0.0)):
            raise ValueError(estimate_precision_error_message)
        precisions_chol = 1.0 / np.sqrt(covariances)
    return precisions_chol


###############################################################################
# Gaussian mixture probability estimators
def _compute_log_det_cholesky(matrix_chol, covariance_type, n_features):
    """Compute the log-det of the cholesky decomposition of matrices.

    Parameters
    ----------
    matrix_chol : array-like
        Cholesky decompositions of the matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)

    covariance_type : {'full', 'tied', 'diag', 'spherical'}

    n_features : int
        Number of features.

    Returns
    -------
    log_det_precision_chol : array-like of shape (n_components,)
        The determinant of the precision matrix for each component.
    """
    if covariance_type == "full":
        n_components, _, _ = matrix_chol.shape
        log_det_chol = np.sum(
            np.log(matrix_chol.reshape(n_components, -1)[:, ::n_features + 1]),
            1)

    elif covariance_type == "tied":
        log_det_chol = np.sum(np.log(np.diag(matrix_chol)))

    elif covariance_type == "diag":
        log_det_chol = np.sum(np.log(matrix_chol), axis=1)

    else:
        log_det_chol = n_features * (np.log(matrix_chol))

    return log_det_chol


def _estimate_log_gaussian_prob(X, means, precisions_chol, covariance_type):
    """Estimate the log Gaussian probability.

    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)

    means : array-like of shape (n_components, n_features)

    precisions_chol : array-like
        Cholesky decompositions of the precision matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)

    covariance_type : {'full', 'tied', 'diag', 'spherical'}

    Returns
    -------
    log_prob : array, shape (n_samples, n_components)
    """
    n_samples, n_features = X.shape
    n_components, _ = means.shape
    # det(precision_chol) is half of det(precision)
    log_det = _compute_log_det_cholesky(precisions_chol, covariance_type,
                                        n_features)

    if covariance_type == "full":
        log_prob = np.empty((n_samples, n_components))
        for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
            y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
            log_prob[:, k] = np.sum(np.square(y), axis=1)

    elif covariance_type == "tied":
        log_prob = np.empty((n_samples, n_components))
        for k, mu in enumerate(means):
            y = np.dot(X, precisions_chol) - np.dot(mu, precisions_chol)
            log_prob[:, k] = np.sum(np.square(y), axis=1)

    elif covariance_type == "diag":
        precisions = precisions_chol**2
        log_prob = (np.sum((means**2 * precisions), 1) -
                    2.0 * np.dot(X, (means * precisions).T) +
                    np.dot(X**2, precisions.T))

    elif covariance_type == "spherical":
        precisions = precisions_chol**2
        log_prob = (np.sum(means**2, 1) * precisions -
                    2 * np.dot(X, means.T * precisions) +
                    np.outer(row_norms(X, squared=True), precisions))
    return -0.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det


class ImprovedGaussianMixture(BaseMixture):
    """Gaussian Mixture.

    Representation of a Gaussian mixture model probability distribution.
    This class allows to estimate the parameters of a Gaussian mixture
    distribution.

    Read more in the :ref:`User Guide <gmm>`.

    .. versionadded:: 0.18

    Parameters
    ----------
    n_components : int, default=1
        The number of mixture components.

    covariance_type : {'full', 'tied', 'diag', 'spherical'}, default='full'
        String describing the type of covariance parameters to use.
        Must be one of:

        'full'
            each component has its own general covariance matrix
        'tied'
            all components share the same general covariance matrix
        'diag'
            each component has its own diagonal covariance matrix
        'spherical'
            each component has its own single variance

    tol : float, default=1e-3
        The convergence threshold. EM iterations will stop when the
        lower bound average gain is below this threshold.

    reg_covar : float, default=1e-6
        Non-negative regularization added to the diagonal of covariance.
        Allows to assure that the covariance matrices are all positive.

    max_iter : int, default=100
        The number of EM iterations to perform.

    n_init : int, default=1
        The number of initializations to perform. The best results are kept.

    init_params : {'kmeans', 'random'}, default='kmeans'
        The method used to initialize the weights, the means and the
        precisions.
        Must be one of::

            'kmeans' : responsibilities are initialized using kmeans.
            'random' : responsibilities are initialized randomly.

    weights_init : array-like of shape (n_components, ), default=None
        The user-provided initial weights.
        If it is None, weights are initialized using the `init_params` method.

    means_init : array-like of shape (n_components, n_features), default=None
        The user-provided initial means,
        If it is None, means are initialized using the `init_params` method.

    precisions_init : array-like, default=None
        The user-provided initial precisions (inverse of the covariance
        matrices).
        If it is None, precisions are initialized using the 'init_params'
        method.
        The shape depends on 'covariance_type'::

            (n_components,)                        if 'spherical',
            (n_features, n_features)               if 'tied',
            (n_components, n_features)             if 'diag',
            (n_components, n_features, n_features) if 'full'

    random_state : int, RandomState instance or None, default=None
        Controls the random seed given to the method chosen to initialize the
        parameters (see `init_params`).
        In addition, it controls the generation of random samples from the
        fitted distribution (see the method `sample`).
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.

    warm_start : bool, default=False
        If 'warm_start' is True, the solution of the last fitting is used as
        initialization for the next call of fit(). This can speed up
        convergence when fit is called several times on similar problems.
        In that case, 'n_init' is ignored and only a single initialization
        occurs upon the first call.
        See :term:`the Glossary <warm_start>`.

    verbose : int, default=0
        Enable verbose output. If 1 then it prints the current
        initialization and each iteration step. If greater than 1 then
        it prints also the log probability and the time needed
        for each step.

    verbose_interval : int, default=10
        Number of iteration done before the next print.

    Attributes
    ----------
    weights_ : array-like of shape (n_components,)
        The weights of each mixture components.

    means_ : array-like of shape (n_components, n_features)
        The mean of each mixture component.

    covariances_ : array-like
        The covariance of each mixture component.
        The shape depends on `covariance_type`::

            (n_components,)                        if 'spherical',
            (n_features, n_features)               if 'tied',
            (n_components, n_features)             if 'diag',
            (n_components, n_features, n_features) if 'full'

    precisions_ : array-like
        The precision matrices for each component in the mixture. A precision
        matrix is the inverse of a covariance matrix. A covariance matrix is
        symmetric positive definite so the mixture of Gaussian can be
        equivalently parameterized by the precision matrices. Storing the
        precision matrices instead of the covariance matrices makes it more
        efficient to compute the log-likelihood of new samples at test time.
        The shape depends on `covariance_type`::

            (n_components,)                        if 'spherical',
            (n_features, n_features)               if 'tied',
            (n_components, n_features)             if 'diag',
            (n_components, n_features, n_features) if 'full'

    precisions_cholesky_ : array-like
        The cholesky decomposition of the precision matrices of each mixture
        component. A precision matrix is the inverse of a covariance matrix.
        A covariance matrix is symmetric positive definite so the mixture of
        Gaussian can be equivalently parameterized by the precision matrices.
        Storing the precision matrices instead of the covariance matrices makes
        it more efficient to compute the log-likelihood of new samples at test
        time. The shape depends on `covariance_type`::

            (n_components,)                        if 'spherical',
            (n_features, n_features)               if 'tied',
            (n_components, n_features)             if 'diag',
            (n_components, n_features, n_features) if 'full'

    converged_ : bool
        True when convergence was reached in fit(), False otherwise.

    n_iter_ : int
        Number of step used by the best fit of EM to reach the convergence.

    lower_bound_ : float
        Lower bound value on the log-likelihood (of the training data with
        respect to the model) of the best fit of EM.

    n_features_in_ : int
        Number of features seen during :term:`fit`.

        .. versionadded:: 0.24

    feature_names_in_ : ndarray of shape (`n_features_in_`,)
        Names of features seen during :term:`fit`. Defined only when `X`
        has feature names that are all strings.

        .. versionadded:: 1.0

    See Also
    --------
    BayesianGaussianMixture : Gaussian mixture model fit with a variational
        inference.

    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.mixture import GaussianMixture
    >>> X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
    >>> gm = GaussianMixture(n_components=2, random_state=0).fit(X)
    >>> gm.means_
    array([[10.,  2.],
           [ 1.,  2.]])
    >>> gm.predict([[0, 0], [12, 3]])
    array([1, 0])
    """
    def __init__(
        self,
        n_components=1,
        *,
        covariance_type="full",
        tol=1e-3,
        reg_covar=1e-6,
        max_iter=100,
        n_init=1,
        init_params="kmeans",
        weights_init=None,
        means_init=None,
        precisions_init=None,
        random_state=None,
        warm_start=False,
        verbose=0,
        verbose_interval=10,
    ):
        super().__init__(
            n_components=n_components,
            tol=tol,
            reg_covar=reg_covar,
            max_iter=max_iter,
            n_init=n_init,
            init_params=init_params,
            random_state=random_state,
            warm_start=warm_start,
            verbose=verbose,
            verbose_interval=verbose_interval,
        )

        self.covariance_type = covariance_type
        self.weights_init = weights_init
        self.means_init = means_init
        self.precisions_init = precisions_init

    def _check_parameters(self, X):
        """Check the Gaussian mixture parameters are well defined."""
        _, n_features = X.shape
        if self.covariance_type not in ["spherical", "tied", "diag", "full"]:
            raise ValueError("Invalid value for 'covariance_type': %s "
                             "'covariance_type' should be in "
                             "['spherical', 'tied', 'diag', 'full']" %
                             self.covariance_type)

        if self.weights_init is not None:
            self.weights_init = _check_weights(self.weights_init,
                                               self.n_components)

        if self.means_init is not None:
            self.means_init = _check_means(self.means_init, self.n_components,
                                           n_features)

        if self.precisions_init is not None:
            self.precisions_init = _check_precisions(
                self.precisions_init,
                self.covariance_type,
                self.n_components,
                n_features,
            )

    def _initialize(self, X, resp, B=None):
        """Initialization of the Gaussian mixture parameters.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        resp : array-like of shape (n_samples, n_components)
        """
        n_samples, _ = X.shape

        weights, means, covariances = _estimate_gaussian_parameters(
            X, resp, self.reg_covar, self.covariance_type, B=B)
        weights /= n_samples

        self.weights_ = weights if self.weights_init is None else self.weights_init
        self.means_ = means if self.means_init is None else self.means_init

        if self.precisions_init is None:
            self.covariances_ = covariances
            self.precisions_cholesky_ = _compute_precision_cholesky(
                covariances, self.covariance_type)
        elif self.covariance_type == "full":
            self.precisions_cholesky_ = np.array([
                linalg.cholesky(prec_init, lower=True)
                for prec_init in self.precisions_init
            ])
        elif self.covariance_type == "tied":
            self.precisions_cholesky_ = linalg.cholesky(self.precisions_init,
                                                        lower=True)
        else:
            self.precisions_cholesky_ = self.precisions_init

    def _m_step(self, X, log_resp, B=None):
        """M step.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        log_resp : array-like of shape (n_samples, n_components)
            Logarithm of the posterior probabilities (or responsibilities) of
            the point of each sample in X.
        """
        n_samples, _ = X.shape
        self.weights_, self.means_, self.covariances_ = _estimate_gaussian_parameters(
            X, np.exp(log_resp), self.reg_covar, self.covariance_type, B=B)
        self.weights_ /= n_samples
        self.precisions_cholesky_ = _compute_precision_cholesky(
            self.covariances_, self.covariance_type)

    def _estimate_log_prob(self, X):
        return _estimate_log_gaussian_prob(X, self.means_,
                                           self.precisions_cholesky_,
                                           self.covariance_type)

    def _estimate_log_weights(self):
        return np.log(self.weights_)

    def _compute_lower_bound(self, _, log_prob_norm):
        return log_prob_norm

    def _get_parameters(self):
        return (
            self.weights_,
            self.means_,
            self.covariances_,
            self.precisions_cholesky_,
        )

    def _set_parameters(self, params):
        (
            self.weights_,
            self.means_,
            self.covariances_,
            self.precisions_cholesky_,
        ) = params

        # Attributes computation
        _, n_features = self.means_.shape

        if self.covariance_type == "full":
            self.precisions_ = np.empty(self.precisions_cholesky_.shape)
            for k, prec_chol in enumerate(self.precisions_cholesky_):
                self.precisions_[k] = np.dot(prec_chol, prec_chol.T)

        elif self.covariance_type == "tied":
            self.precisions_ = np.dot(self.precisions_cholesky_,
                                      self.precisions_cholesky_.T)
        else:
            self.precisions_ = self.precisions_cholesky_**2

    def _n_parameters(self):
        """Return the number of free parameters in the model."""
        _, n_features = self.means_.shape
        if self.covariance_type == "full":
            cov_params = self.n_components * n_features * (n_features +
                                                           1) / 2.0
        elif self.covariance_type == "diag":
            cov_params = self.n_components * n_features
        elif self.covariance_type == "tied":
            cov_params = n_features * (n_features + 1) / 2.0
        elif self.covariance_type == "spherical":
            cov_params = self.n_components
        mean_params = n_features * self.n_components
        return int(cov_params + mean_params + self.n_components - 1)

    def bic(self, X):
        """Bayesian information criterion for the current model on the input X.

        Parameters
        ----------
        X : array of shape (n_samples, n_dimensions)
            The input samples.

        Returns
        -------
        bic : float
            The lower the better.
        """
        return -2 * self.score(X) * X.shape[0] + self._n_parameters() * np.log(
            X.shape[0])

    def aic(self, X):
        """Akaike information criterion for the current model on the input X.

        Parameters
        ----------
        X : array of shape (n_samples, n_dimensions)
            The input samples.

        Returns
        -------
        aic : float
            The lower the better.
        """
        return -2 * self.score(X) * X.shape[0] + 2 * self._n_parameters()

In [3]:
from collections import defaultdict
import numpy as np


def read_markers(
    fname,
    markers_ec=defaultdict(list),
    celltype=defaultdict(),
    marker_genes=defaultdict(),
):
    with open(fname, "r") as f:
        for idx, line in enumerate(f.readlines()):
            ct, genes = line.strip().split("\t")
            celltype[ct] = idx

            # two things
            # 1. make marker_genes list
            # 2. make markers_ec
            for g in genes.split(","):
                gidx = len(marker_genes)

                # check if the gene has been added already
                if g in marker_genes.keys():  # gene repeated
                    gidx = marker_genes[g]
                else:
                    marker_genes[g] = gidx

                # for the cell type index, add the marker gene index
                markers_ec[celltype[ct]].append(marker_genes[g])

            # sort the marker genes
            markers_ec[celltype[ct]] = sorted(markers_ec[celltype[ct]])


def read_int_list(fname, lst=[]):
    with open(fname) as f:
        for idx, i in enumerate(f.readlines()):
            lst.append(int(i.strip()))


def read_str_list(fname, lst=list):
    with open(fname, "r") as f:
        for idx, line in enumerate(f.readlines()):
            lst.append(line.strip())


def write_list(fname, lst=list):
    with open(fname, "w") as f:
        for idx, ele in enumerate(lst):
            f.write(f"{ele}\n")


def write_markers(fname, markers):
    with open(fname, "w") as f:
        for k, v in markers.items():
            f.write(f"{k}\t")
            n = len(v)
            for idx, i in enumerate(v):
                f.write(f"{i}")
                if idx < n - 1:
                    f.write(",")
            f.write("\n")


def map_dict_list_keys_values(dct, k_lst, v_lst, nd=defaultdict(list)):
    for k, v in dct.items():
        nd[k_lst[k]] = [v_lst[i] for i in v]


def map_dict_list_keys(dct, k_lst, v_lst, nd=defaultdict(list)):
    for k, v in dct.items():
        nd[k_lst[k]] = v


def map_dict_list_values(dct, k_lst, v_lst, nd=defaultdict(list)):
    for k, v in dct.items():
        nd[k] = [v_lst[i] for i in v]


def read_markers_ec(fname, markers_ec=defaultdict(list)):
    with open(fname, "r") as f:
        for idx, line in enumerate(f.readlines()):
            ct_id, gene_ids = line.strip().split("\t")
            markers_ec[int(ct_id)] = [int(i) for i in gene_ids.split(",")]


def read_markers_str(fname, markers=defaultdict(list)):
    with open(fname, "r") as f:
        for idx, line in enumerate(f.readlines()):
            ct_id, gene_ids = line.strip().split("\t")
            markers[ct_id] = [i for i in gene_ids.split(",")]


def sanitize_mtx(mtx):
    cell_count_mask = mtx.sum(1) > 0  # count for each cell
    gene_count_mask = mtx.sum(0) > 0  # count for each gene

    genes_detected_mask = (mtx > 0).sum(1) > 0  # n genes per cell
    cells_detected_mask = (mtx > 0).sum(0) > 0  # n cells per gene
    row_mask = np.logical_and(cell_count_mask, genes_detected_mask)
    col_mask = np.logical_and(gene_count_mask, cells_detected_mask)

    return (row_mask, col_mask)


def drop_markers(markers_ec, drop_ids):
    if len(drop_ids) == 0:
        return

    for k, v in markers_ec.items():

        gidx = len(v) - 1
        while gidx > -1:
            mg = markers_ec[k][gidx]

            if mg in drop_ids:
                markers_ec[k].pop(gidx)
            else:
                to_sub = 0
                for d in drop_ids:
                    if d < mg:
                        to_sub += 1
                markers_ec[k][gidx] -= to_sub
            gidx -= 1


# testing data
# drop_ids = set([2, 3, 34, 42])
# truth = {0: [0, 1, 2, 3, 4, 5, 6, 7, 8],
#         1: [7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
#         2: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
#         3: [0, 3,  6,  7, 24, 29, 30, 31, 32, 33],
#         4: [0, 4,  5,  6,  7, 18, 23, 24, 30, 31, 34],
#         5: [2, 22, 23, 24, 30, 35, 36, 37, 38, 39],
#         6: [0, 3,  4,  6,  7, 24, 30, 31, 32, 40]}
# drop_markers(markers_ec, set(drop_genes))
# markers_ec == truth


def get_marker_centroids(X, markers_ec, method="mean"):
    n_clusters = len(list(markers_ec.keys()))
    _, n_features = X.shape

    marker_centroids = np.ones((n_clusters, n_features)) * 1e-5

    for k, v in markers_ec.items():
        submx = X[:, v]
        if method == "max":
            repl = submx.max(0)
        else:
            repl = submx.mean(0)
        marker_centroids[k][v] = repl
    return marker_centroids


def get_centroids(X, z):
    clusters = np.sort(np.unique(z))

    (n_clusters,) = clusters.shape
    _, n_features = X.shape

    centroids = np.ones((n_clusters, n_features))
    for i, g in enumerate(clusters):
        centroids[i] = X[np.where(z == g)[0]].mean(0)
    return centroids


In [4]:
#@title import
import os
from ipfn import ipfn
import matplotlib.pyplot as plt
from sklearn.metrics import rand_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
import json
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from collections import defaultdict
from scipy.io import mmread, mmwrite
from scipy.sparse import csr_matrix
from sklearn.neighbors import KDTree
from scipy.stats import entropy
from itertools import combinations
import sys
import gzip
from scipy.stats import entropy
from sklearn.mixture import GaussianMixture
from kb_python.utils import import_matrix_as_anndata
import scipy as sp

def nd(arr):
    return np.asarray(arr).reshape(-1)

def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

from upsetplot import from_memberships, plot as upsetplot, from_contents

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [231]:
sample = "293T"
observation = "obs2"

out_name = "tags_out"
file_base = "cells_x_features"

base_data = f"azucar/data/{sample}/{observation}"
base_mark = f"azucar/analysis/{sample}/{observation}/assign"

in_matrix_fn  = os.path.join(base_data, "matrix.mtx.gz")
in_barcodes_fn   = os.path.join(base_data, "barcodes.txt.gz")
in_genes_fn   = os.path.join(base_data, "genes.txt.gz")

markers_ec_fn = os.path.join(base_mark, "markers.txt")

In [232]:
adata = import_matrix_as_anndata(in_matrix_fn, in_barcodes_fn, in_genes_fn)
adata

AnnData object with n_obs × n_vars = 3509 × 5

In [233]:
def do_pf(mtx, sf = None):
    pf = mtx.sum(axis=1).A.ravel()
    if not sf:
        sf = pf.mean()
    pf = sp.sparse.diags(sf/pf) @ mtx
    return pf

def norm_pf_log_pf(mtx):
    pf_log_pf = do_pf(np.log1p(do_pf(mtx)))
    return pf_log_pf

def norm_pf_sqrt(mtx):
    pf_sqrt = np.sqrt(do_pf(mtx))
    return pf_sqrt

def norm_sqrt_pf(mtx):
    sqrt_pf = do_pf(np.sqrt(mtx))
    return sqrt_pf

def norm_sqrt(mtx):
    sqrt = np.sqrt(mtx)
    return sqrt

def norm_pf_log(mtx):
    pf_log = np.log1p(do_pf(mtx))
    return pf_log

def norm_pf_sqrt_pf(mtx):
    pf_sqrt_pf = do_pf(np.sqrt(do_pf(mtx)))
    return pf_sqrt_pf


NORM = {
    "PF": do_pf,
    "PFlog1pPF": norm_pf_log_pf,
    "PFsqrtPF" : norm_pf_sqrt_pf,
    "sqrtPF": norm_pf_sqrt,
    "log1pPF": norm_pf_log,
    "sqrt": norm_sqrt,
    "PFsqrt": norm_sqrt_pf
}

In [234]:
# normalize
adata.layers["raw"] = adata.X

adata.layers["PFlog1pPF"] = NORM["PFlog1pPF"](adata.X)
adata.layers["PFsqrtPF"] = NORM["PFsqrtPF"](adata.X)
adata.layers["sqrtPF"] = NORM["sqrtPF"](adata.X)
adata.layers["PFsqrt"] = NORM["PFsqrt"](adata.X)
adata.layers["sqrt"] = NORM["sqrt"](adata.X)
adata.layers["log1pPF"] = NORM["log1pPF"](adata.X)
adata.layers["PF"] = NORM["PF"](adata.X)

In [235]:
marker_genes = ["mtag1","mtag2","mtag3","mtag4"]
groups = ["no_sugar", "control", "tmg"]

markers_ec = {
    0: [0, 2], # no_sugar: mtag1,mtag3
    1: [2, 3], # control:  mtag3,mtag4
    2: [1, ]   # tmg:      mtag2,mtag3
}

markers = {
    "no_sugar": ["mtag1", "mtag3"],
    "control":  ["mtag3", "mtag4"],
    "tmg":      ["mtag2", "mtag3"]
}

In [236]:
!cat $markers_ec_fn

no_sugar	mtag1,mtag3
control	mtag3,mtag4
tmg	mtag2,mtag3


In [237]:
for k, v in markers_ec.items():
    print(groups[k], [marker_genes[i] for i in v])

no_sugar ['mtag1', 'mtag3']
control ['mtag3', 'mtag4']
tmg ['mtag2']


In [238]:
markers

{'no_sugar': ['mtag1', 'mtag3'],
 'control': ['mtag3', 'mtag4'],
 'tmg': ['mtag2', 'mtag3']}

In [239]:
subset = adata[:, adata.var.index.isin(marker_genes)].copy()

In [240]:
subset.var.index.values == marker_genes

array([ True,  True,  True,  True])

## `mx assign`

In [241]:
def mx_assign(G, barcodes, genes, groups, markers_ec):
    n_clusters = len(markers_ec.keys())
    n_samples, n_features = G.shape
    # print(n_clusters, *G.shape, sep=", ")

    # have to initialize the clusters by first mean centering alternative is to zscore the means

    X_mean = G.mean(0)
    X_init = (G - X_mean)
    centroids_init = get_marker_centroids(X_init, markers_ec, "max")

    tree = KDTree(centroids_init, metric="euclidean")
    nearest_dist, nearest_ind = tree.query(X_init, k=1)

    # assign cells to clusters
    p = 1
    resp = np.ones((n_samples, n_clusters)) * (1 - p) / (n_clusters - 1)
    resp[np.arange(n_samples), nearest_ind.flatten()] = p

    # initialize params
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps

    # then once we have the means, add the previously subtracted means back
    means_init = np.dot(resp.T, X_init) / nk[:, np.newaxis]
    means_init += X_mean

    # alternative to uniform weights is nk / n_samples (using the new assignments)
    uniform_weights = np.array([1. / n_clusters] * n_clusters)

    # alternative is to compute precisions by first doing M-step to get gaus params
    identity_precisions = np.repeat(
      np.array([np.eye(G.shape[1])]), n_clusters, 0)


    gmm_params = {
      "n_components": n_clusters,
      "means_init": None,  # to be added
      "weights_init": None,  # to be added
      "precisions_init": None,  # to be added
      "random_state": 0,
      "reg_covar": 1e-8,
      "verbose": 2,
      "n_init": 1,
      "max_iter": 1000,
      "tol": 1e-4,
      "init_params": "random",
      "covariance_type": "full"
    }

    params = {
      **gmm_params,
      "means_init": means_init,  # centroids,
      "weights_init": uniform_weights,
      "precisions_init": identity_precisions
    }

    # actually do the GMM
    gmm = ImprovedGaussianMixture(**params)

    # print(method, json.dumps(params, indent=4, default=str))

    labels = gmm.fit_predict(G, B=markers_ec)
    means = gmm.means_
    prob = gmm.predict_proba(G)
    ent = entropy(prob, axis=1)

    # make df
    df = pd.DataFrame(G, columns=[f"{i}" for i in genes])

    df["label_id"] = labels
    df["label"] = df["label_id"].map({i:groups[i] for i in range(len(groups))})
    df["ent"] = ent
    df.index = barcodes

    for idx, p in enumerate(prob.T):
        df[f"mahalanobis_{idx}"] = p

    return df

def mx_assign_fn(matrix_fn, barcodes_fn, genes_fn, markers_ec_fn, groups_fn):
    # mx assign assumes matrix has been filtered and that genes are ordered by their numbering in matrix_ec
    groups = []
    read_str_list(groups_fn, groups)

    genes = []
    read_str_list(genes_fn, genes)

    barcodes = []
    read_str_list(barcodes_fn, barcodes)

    markers_ec = defaultdict(list)
    read_markers_ec(markers_ec_fn, markers_ec)

    # read in gene count matrix
    G = mmread(matrix_fn).toarray()
    
    return mx_assign(G, barcodes, genes, groups, markers_ec)

In [242]:
marker_genes

['mtag1', 'mtag2', 'mtag3', 'mtag4']

In [267]:
layer = "PF"

X = subset.layers[layer].A
mtag2_idx = np.where(np.array(marker_genes) == "mtag2")[0]
mtag3_idx = np.where(np.array(marker_genes) == "mtag3")[0]
# X[:,mtag2_idx] = X[:,mtag2_idx]*3
# X[:,mtag3_idx] = X[:,mtag3_idx]

df = mx_assign(X, 
               subset.obs.index.values, 
               subset.var.index.values, 
               groups, 
               markers_ec)
df["dbco"] = adata.layers[layer].A[:,adata.var.index.values == "dbco"]

raw = adata.X.A
for idx, gvec in enumerate(adata.X.A.T):
    df[f"{adata.var.index.values[idx]}_raw"] = gvec

Initialization 0
  Iteration 10	 time lapse 0.03019s	 ll change 0.01145
  Iteration 20	 time lapse 0.02782s	 ll change -0.00116
  Iteration 30	 time lapse 0.02681s	 ll change -0.00043
Initialization converged: True	 time lapse 0.09046s	 ll -21.52169


In [268]:
X.sum(0).astype(int).reshape(-1,1)

array([[ 371929],
       [ 196217],
       [1040793],
       [ 274092]])

In [269]:
df.label.value_counts()

control     1730
no_sugar    1521
tmg          258
Name: label, dtype: int64

In [270]:
df.groupby("label")[["dbco"]].mean().sort_values("dbco")

,dbco
label,
no_sugar,6.374557
tmg,9.672799
control,14.144555


In [271]:
df.groupby("label")[["dbco_raw"]].mean().sort_values("dbco_raw")

,dbco_raw
label,
no_sugar,11.448389
control,17.905203
tmg,39.062016


In [272]:
df.groupby("label").agg({
      "label": "count",
      "dbco" : "mean",
      "mtag1": "mean", 
      "mtag2": "mean",
      "mtag3": "mean",
      "mtag4": "mean"
      }).loc[groups].astype(float)

,label,dbco,mtag1,mtag2,mtag3,mtag4
label,,,,,,
no_sugar,1521.0,6.374557,206.254898,6.820661,304.132507,23.494879
control,1730.0,14.144555,32.365852,46.170990,319.589142,134.806992
tmg,258.0,9.672799,8.616586,410.726227,98.139687,19.922215


In [273]:
df.groupby("label").agg({
      "label": "count",
       "dbco_raw": "mean",
      "mtag1_raw": "mean", 
      "mtag2_raw": "mean",
      "mtag3_raw": "mean",
      "mtag4_raw": "mean"
      }).loc[groups].astype(int)

,label,dbco_raw,mtag1_raw,mtag2_raw,mtag3_raw,mtag4_raw
label,,,,,,
no_sugar,1521,11,165,8,368,17
control,1730,17,29,40,325,156
tmg,258,39,1,187,28,4


In [274]:
df.filter(like="_raw").sum(0).astype(int)

mtag1_raw     303371
mtag2_raw     130006
mtag3_raw    1129765
mtag4_raw     298086
dbco_raw       58467
dtype: int64

In [175]:
# how do you know which mean goes with which group?

In [148]:
markers

{'no_sugar': ['mtag1', 'mtag3'],
 'control': ['mtag3', 'mtag4'],
 'tmg': ['mtag2', 'mtag3']}

In [279]:
# make violin plot and try other datasets

In [278]:
df.groupby("label")["mtag1"].apply(list)

label
control     [28.41961097717285, 15.829790115356445, 26.546...
no_sugar    [24.42310333251953, 218.8310089111328, 0.0, 12...
tmg         [21.595165252685547, 0.0, 35.79011917114258, 0...
Name: mtag1, dtype: object

## `mx diff`

In [26]:
#@title mx diff

from scipy.stats import ks_2samp, ttest_ind
import numpy as np
import pandas as pd
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')

import numpy as np
from statsmodels.stats.multitest import multipletests

def split_by_target(mat, targets, target, axis=0):
    """
    Split the rows of mat by the proper assignment
    
    mat = ndarray
    targets, length is equal to number of components (axis=0) or features (axis=1)
    target is a singular element from unique(assignments/features) 
    """

    if axis==0 and len(targets) != mat.shape[axis]: return -1
    if axis==1 and len(targets) != mat.shape[axis]: return -1
        
    mask = targets == target
    
    if axis==0:
        t_mat = mat[mask]  # target matrix
        c_mat = mat[~mask] # complement matrix
    elif axis==1:
        t_mat = mat[:, mask]  # target matrix
        c_mat = mat[:, ~mask] # complement matrix
    
    return (t_mat, c_mat)

def correct_pval(pval, nfeat, method):
    if method == 'bonferroni':
        pval_corr = np.minimum(pval*nfeat[:,None], 1.0)
    return pval_corr

def nd(arr):
    return np.asarray(arr).reshape(-1)
def correct_pvals(pvals):
    r = np.empty(np.shape(pvals))
    r[:] = np.nan
    for pidx, p in enumerate(pvals):
        mask = ~np.isnan(p)


    return r


def dexpress(mat, components, features, assignments, **kwargs):
    """
    mat: ndarray
    components: nparray
    features: nparray
    assignments: nparray, length is equal to length of components
    """
    nan_cutoff = kwargs.get("nan_cutoff", 0)
    
    unique = np.unique(assignments)
        
    ncomp = components.shape[0]
    nfeat = features.shape[0]
    nuniq = unique.shape[0]
    
    pval = np.empty((nuniq, nfeat)) # unique assignments by features 
    pval_corr = np.empty((nuniq, nfeat)) # unique assignments by features 
    stat = np.empty((nuniq, nfeat)) # unique assignments by features 
    es = np.empty((nuniq, nfeat)) # unique assignments by features 
    nfeatures = np.empty((nuniq))
    
    pval.fill(np.nan)
    pval_corr.fill(np.nan)
    stat.fill(np.nan)
    es.fill(np.nan)
    nfeatures.fill(np.nan)
    

    for tidx, t in enumerate(unique):
        logging.info("{} of {} assignments: {}".format(tidx+1, nuniq, t))

        t_mat, c_mat = split_by_target(mat, assignments, t, axis=0)
        
        pvalue = np.nan
        statistic = np.nan
        effect_size = np.nan

        # Keep only the genes for which it is in at least nan_cutoff*ncellsinclusters
        mask = np.asarray((t_mat>0).sum(axis=0) > nan_cutoff*t_mat.shape[0]).reshape(-1)

        t_mat = t_mat[:, mask]
        c_mat = c_mat[:, mask]
        # added

        # added
        if mask.sum()>0:
            test = ttest_ind(t_mat, c_mat, nan_policy="propagate", equal_var=False)
            pvalue = test.pvalue
            statistic = test.statistic
            effect_size = nd(t_mat.mean(axis=0)) - nd(c_mat.mean(axis=0))
            nfeatures[tidx] = mask.sum()

        pval[tidx,:][mask] = pvalue
        stat[tidx,:][mask] = statistic
        es[tidx,:][mask] = effect_size
    return (pval, stat, es, nfeatures)

def make_table(assignments, features, p_raw, p_corr, es):
    unique = np.unique(assignments)
    markers = pd.DataFrame(index=unique, columns=["name", "p_raw", "p_corr", "es"])

    for uidx, u in enumerate(unique):
        markers.loc[u]["name"] = features
        markers.loc[u]["p_raw"] = p_raw[uidx]
        markers.loc[u]["p_corr"] = p_corr[uidx]
        markers.loc[u]["es"] = es[uidx]

    markers = markers.apply(pd.Series.explode).reset_index()
    markers = markers[markers.notnull()["p_raw"]]
    markers = markers.rename(columns={"index": "group_id"})
    return markers

def mx_diff(matrix_fn, barcodes_fn, genes_fn, assignments_fn):
  mat         = np.log1p(mmread(matrix_fn).toarray())
  
  components = []
  read_str_list(barcodes_fn, components)
  components = nd(components)

  features = []
  read_str_list(genes_fn, features)
  features = nd(features)
  # assignments = nd(read_str_list(assignments_fn))
  # change to read in assignments from file

  assignments = assignments_fn

  # mat           = np.log1p(mtx.todense())
  # components    = bcs.bcs.values
  # features      = genes.genes.values
  # assignments   = bcs.cellltype_id.values

  unique = np.unique(assignments)
  nan_cutoff = 0.1 # of elements in cluster
  corr_method = "bonferroni"


  p_raw, stat, es, nfeat = dexpress(mat, components, features, assignments, nan_cutoff=nan_cutoff)
  p_raw = p_raw/2
  p_corr = correct_pval(p_raw, nfeat, corr_method)

  s = stat

  markers_gene = make_table(assignments, features, p_raw, p_corr, es)

  # convert the 0 pvalues to the smallest possible float
  markers_gene["p_corr"][markers_gene.eval("p_corr == 0").values] = sys.float_info.min
  # markers_gene = markers_gene.query("es > 0")
  return markers_gene

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Real data

In [62]:
sample = "293T"
observation = "obs4"

base_data = f"azucar/analysis/{sample}/{observation}/out"
base_mark = f"azucar/analysis/{sample}/{observation}/assign"

matrix_fn  = os.path.join(base_data, "matrix.mtx")
genes_fn   = os.path.join(base_data, "genes.txt")
barcodes_fn   = os.path.join(base_data, "barcodes.txt")

!gunzip $base_data/*.gz

# All markers

1. Review normalization procedure.
2. Check covariance usage with gmm
3. Why is mahalanobis distance index inconsistent with the label?

In [66]:
#@title full workflow
def main(matrix_fn, barcodes_fn, genes_fn, markers_fn):
  
  # drop barcodes and genes that sum to zero, update barcodes and genes file
  mx_sanitize(matrix_fn, barcodes_fn, genes_fn, 
              "./san.matrix.mtx", 
              "./san.barcodes.txt", 
              "./san.genes.txt")
  
  # knee plot gmm filter
  mx_filter("./san.matrix.mtx",
            "./san.barcodes.txt",
            "./san.fil.matrix.mtx", 
            "./san.fil.barcodes.txt",
            n_comps=3,
            n_iter=1)

  dbco = mmread("./san.fil.matrix.mtx").toarray()[:,-1]

  # normalize matrix (log1p -> ipf)
  mx_norm("./san.fil.matrix.mtx", 
          "./san.log1p.matrix.mtx", how="log1p")
  
  mx_norm("./san.log1p.matrix.mtx", 
          "./san.norm.matrix.mtx", how="ipf")

  dbco_norm = mmread("./san.norm.matrix.mtx").toarray()[:,-1]

  # index the markers -> markers.ec marker_genes.txt groups.txt
  mx_index(markers_fn, 
          "./markers.ec", 
          "./groups.txt", 
          "./marker_genes.txt")

  # get the gene ids -> select.txt (selects in order of markers.ec)
  mx_select(markers_fn, 
            "./san.genes.txt", 
            "./select.txt")

  # extract elements from matrix that are of interest, rows / columns (with associated metadata)
  mx_extract("san.norm.matrix.mtx", 
            "san.genes.txt", 
            "select.txt", 
            "san.norm.extr.matrix.mtx", 
            "san.extr.genes.txt", axis=1)

  # perform assignments with markers and matrices
  df = mx_assign("san.norm.extr.matrix.mtx", 
                "san.fil.barcodes.txt", 
                "san.extr.genes.txt", 
                "markers.ec", 
                "groups.txt")
  print("shape: ", df.shape)
  
  # original counts (this is the same as make df above)
  raw_mtx = mmread("./san.fil.matrix.mtx").toarray()
  raw_genes = []
  read_str_list("./san.genes.txt", raw_genes)
  for idx, v in enumerate(raw_mtx.T):
      df[f"{raw_genes[idx]}"] = v # np.exp(v) - 1

  df["dbco"] = dbco
  df["dbco_ipf"] = dbco_norm

  # for the np log
  df.loc[df['ent'] == 0.0, 'ent'] = np.finfo(float).tiny

  ## find entropy cutoff
  adj_ent = -np.log(df["ent"].values)

  u, xx, v = knee(np.log1p(adj_ent.reshape(-1,1)), axis=1)
  (cutoff, ncells) = gmm(xx, v, n_iter=1, n_comps=3)
  cutoff = np.exp(cutoff) - 1 # adjust (since we log1p it)
  ent_cutoff = np.exp(-cutoff)

  print(ncells, ent_cutoff)

  ## look at count distribution
  return (df, ent_cutoff)#.query(f"ent < {ent_cutoff}")

In [67]:
!rm *.txt *.mtx *.ec

In [68]:
# full tags
markers_fn = os.path.join(base_mark, "markers.txt")
!cat $markers_fn

df, ent_cutoff = main(matrix_fn, barcodes_fn, genes_fn, markers_fn)

no_sugar	mtag1,mtag3
control	mtag3,mtag4
tmg	mtag2,mtag3
Filtered to 95 cells with at least 7,221 UMIs.
Initialization 0
  Iteration 10	 time lapse 0.01594s	 ll change 0.00164
  Iteration 20	 time lapse 0.01265s	 ll change 0.00014
Initialization converged: True	 time lapse 0.03081s	 ll -5.69168
shape:  (95, 10)
42 4.9492937509968145e-08


In [69]:
df.groupby("label").agg({
      "mtag1_ipf": "mean", 
      "mtag2_ipf": "mean",
      "mtag3_ipf": "mean",
      "mtag4_ipf": "mean",
      "dbco_ipf": "mean",
      "label": "count"
      }).astype(float)

,mtag1_ipf,mtag2_ipf,mtag3_ipf,mtag4_ipf,dbco_ipf,label
label,,,,,,
control,5.041637,2.972566,8.170634,8.534180,5.635078,34.0
no_sugar,7.963705,2.944335,8.686129,5.610813,5.149069,42.0
tmg,6.640304,2.921285,6.621226,8.238353,5.932934,19.0


In [ ]:
!rm *.txt *.mtx *.ec

In [ ]:
# slim tags
!sed 's/mtag2,//' $markers_fn > $base_mark/markers_slim.txt
markers_slim_fn = os.path.join(base_mark, "markers_slim.txt")

!cat $markers_slim_fn

df_slim, ent_cutoff_slim = main(matrix_fn, barcodes_fn, genes_fn, markers_slim_fn)

In [ ]:
df_slim.query(f"ent < {ent_cutoff_slim}").groupby("label").agg({
      "mtag1_ipf": "mean", 
      "mtag3_ipf": "mean",
      "mtag4_ipf": "mean",
      "dbco_ipf": "mean",
      "label": "count"
      }).astype(float)

!cat markers_fn

In [ ]:
mdf = mx_diff("san.norm.extr.matrix.mtx", 
              "san.barcodes.txt", 
              "san.extr.genes.txt", 
              df["label"].values)

def convert_mtx(*x):
  # x is a pandas series with multiple values
  # but because our matrix has unique group, tag pairs
  # there is only one element in the series
  return x[0].values[0]

es = mdf.groupby(["group_id", "name"])["es"].apply(convert_mtx).unstack()
pv = mdf.groupby(["group_id", "name"])["p_corr"].apply(convert_mtx).unstack()

xa, ya = np.where(es.values > 0)
check = defaultdict(list)
conditions = es.index.values
tags = es.columns.values
for i, j in zip(xa, ya):
  check[conditions[i]].append(tags[j])

markers = defaultdict(list)
read_markers_str(markers_fn, markers)

# number in true only, number in intersection, number in check only
for ct, gs in markers.items():
  tm = markers[ct]
  cm = check.get(ct, [])

  left  = np.setdiff1d  (tm, cm).shape[0]
  itx   = np.intersect1d(tm, cm).shape[0]
  right = np.setdiff1d  (cm, tm).shape[0]

  print(f"{left, itx, right} \t {ct}")


print(es)
print(pv)

In [ ]:
!cat $markers_fn

In [ ]:
# check mahalanobis index match

In [ ]:
fig, axs = plt.subplots(figsize=(10,3*3), nrows=3, constrained_layout=True)

gb = df.query(f"ent < {ent_cutoff}").groupby("label")
for ax, (cond, g) in zip(axs, gb):
  d = g.filter(like="_ipf")
  
  labels = d.columns.values
  ticks = np.arange(d.shape[1])

  x = np.repeat([np.arange(d.shape[1])], d.shape[0], axis=0) + np.random.normal(0, 0.1, d.shape)
  y = d.values
  c = np.repeat([g["ent"].values.T], 5, 0).T
  argsort = np.argsort(c[:,0])
  ax.scatter(x[argsort], y[argsort], alpha=0.5, c = c[argsort], s=5)

  ax.set(**{
      "xticks": ticks,
      "xticklabels": labels,
      "yscale": "symlog",
      "title": f"{cond}: {','.join(markers[cond])}",
      "xlabel": "tag",
      "ylabel": "UMI counts"
  })

fig.show()

In [ ]:
!cat $markers_fn

In [ ]:
!rm *.txt *.mtx *.ec

In [ ]:
# try making a cell x (summed gene per condition) matrix

# Manual assignment

In [ ]:
out_

In [ ]:
## John Assignment

# load data
mtx = mmread("./san.matrix.mtx").toarray()

bcs = []
read_str_list("./san.barcodes.txt", bcs)

genes = []
read_str_list("./san.genes.txt", genes)

# make df
m = pd.DataFrame(mtx, index = bcs, columns=genes)

# John assignment preprocessing
john = m.div(m.sum(axis=0), axis=1) # normalize gene to be same
john["mtag3"] = john["mtag3"] * 3.
john *= 100000
john.head()

# assignment procedure
def john_assign(row):
  thresh = 40

  # try all conditions
  # where tags for conditions are high
  # and non-tags are low
  # assign
  label = []
  for c, v in markers.items():
    other = list(set(genes) - set(v))

    other.remove("dbco")
    c1 = row[v[0]] > thresh
    c2 = row[v[1]] > thresh
    c3 = row[other[0]] < thresh
    c4 = row[other[1]] < thresh

    if (c1&c2) & (c3&c4):
      label.append(c)
  return label

blah = john.apply(john_assign, axis=1)
d = blah[blah.map(lambda d: len(d) > 0)]
d = d.map(lambda x: x[0])
john["label"] = john.index.map(d)
john = john.dropna()

In [167]:
in_matrix_fn  = os.path.join(base_data,    f"{out_name}/counts_unfiltered/{file_base}.mtx")
in_barcodes_fn   = os.path.join(base_data, f"{out_name}/counts_unfiltered/{file_base}.barcodes.txt")
in_genes_fn   = os.path.join(base_data,    f"{out_name}/counts_unfiltered/{file_base}.genes.txt")

array(['AAACCCAAGAAACCCG', 'AAACCCAAGAACCCGA', 'AAACCCAAGACCCGCT', ...,
       'TTTGTTGTCTGGCCTC', 'TTTGTTGTCTTAGCAG', 'TTTGTTGTCTTAGCTT'],
      dtype=object)

In [174]:
## Top two assignment

# load in data
mtx  = mmread(raw_fn).toarray()[:,:-1]
dbco = mmread(raw_fn).toarray()[:,-1]

bcs = pd.read_csv(in_barcodes_fn, header=None).values.ravel()

genes = list(pd.read_csv(in_genes_fn, header=None).values.ravel())
genes.remove("dbco")
genes = np.asarray(genes)

# make df
man = pd.DataFrame(mtx, index=bcs, columns=genes)

# pick top two expressed barcodes and assign based on map
bcmap = {
    (1,3): "no_sugar",
    (3,4): "control",
    (2,3): "tmg"
}
ind = np.sort(np.argpartition(mtx, -2)[:,-2:], axis=1) + 1
assn = [bcmap.get(tuple(i), None) for i in ind]
man["dbco"] = dbco
man["label"] = assn
man["total_counts"] = man.filter(like="mtag").sum(1)

In [178]:
man.query("total_counts>100")

,mtag1,mtag2,mtag3,mtag4,dbco,label,total_counts
AAACCCAGTAGTAAGT,7.0,0.0,81.0,22.0,3.0,control,110.0
AAACGAAAGAAGTCAT,8.0,2.0,80.0,11.0,2.0,control,101.0
AAACGAACACCGTGAC,6.0,2.0,96.0,20.0,1.0,control,124.0
AAACGAAGTCACTTAG,10.0,7.0,187.0,13.0,7.0,control,217.0
AAACGAAGTCCAGTTA,10.0,4.0,105.0,20.0,1.0,control,139.0
...,...,...,...,...,...,...,...
TTTGGAGGTTTCCATT,143.0,5.0,896.0,24.0,12.0,no_sugar,1068.0
TTTGGAGTCAGTGCGC,4.0,40.0,45.0,16.0,2.0,tmg,105.0
TTTGGTTAGCCTTCTC,10.0,2.0,164.0,18.0,2.0,control,194.0
TTTGGTTGTAGATCCT,10.0,4.0,87.0,9.0,2.0,no_sugar,110.0


In [180]:
man.query("total_counts>0").groupby("label").agg({
      "label": "count",
       "dbco": "mean",
      "mtag1": "mean", 
      "mtag2": "mean",
      "mtag3": "mean",
      "mtag4": "mean"
      }).loc[groups].astype(int)

,label,dbco,mtag1,mtag2,mtag3,mtag4
label,,,,,,
no_sugar,9820,3,29,2,86,6
control,128104,1,2,0,26,6
tmg,1562,15,13,50,223,9


In [ ]:
# gmm v john
common = np.intersect1d(john.index.values, df.index.values)
print((df.loc[common].label.values == john.loc[common].label.values).sum(), "out of", common.shape[0])

In [ ]:
# man v john
common = np.intersect1d(john.index.values, man.index.values)
print((man.loc[common].label.values == john.loc[common].label.values).sum(), "out of", common.shape[0])

In [ ]:
# man v df
common = np.intersect1d(df.index.values, man.index.values)
print((man.loc[common].label.values == df.loc[common].label.values).sum(), "out of", common.shape[0])

In [ ]:
john.groupby("label")["dbco"].mean()

In [ ]:
man.loc[common].groupby("label")["dbco"].mean()

In [ ]:
# filtering is important!

In [ ]:
#@title old code

# # column indices to select from gene matrix
# select_fn = "select.txt"
# sel = []
# read_int_list(select_fn, sel)

# # the group names
# groups_fn = "groups.txt"
# groups = []
# read_str_list(groups_fn, groups)

# # the barcode names
# barcodes = []
# read_str_list(barcodes_fn, barcodes)

# # the gene names
# genes = []
# read_str_list(genes_fn, genes)

# # the marker gene names that are being selected for
# # NOTE: sel is the index "vector" for marker_genes
# marker_genes_fn = "marker_genes.txt"
# marker_genes = []
# read_str_list(marker_genes_fn, marker_genes)

# # markers.ec, maps groups (indices) to marker genes (indices)
# markers_ec_fn = "markers.ec"
# markers_ec = defaultdict(list)
# read_markers_ec(markers_ec_fn, markers_ec)

# # named groups to named marker genes (value version of markers_ec)
# markers = defaultdict(list)
# map_dict_list_keys_values(markers_ec, groups, marker_genes, markers)

# # read in matrix and select columns and write back to disc
# M = mmread(matrix_fn).toarray()

# # sanitize gene count matrix (remove cells / genes) and remove genes from  marker_ec
# row_mask, col_mask = sanitize_mtx(M)
# barcodes = np.array(barcodes)[row_mask]

# drop_genes = np.arange(M.shape[1])[~col_mask]
# drop_markers(markers_ec, set(drop_genes))

# mtx = M[row_mask][:,col_mask].astype(int)
# mtx_ipf = do_ipf(mtx.copy())
# matrix_select_fn = "matrix_select_ipf.mtx"

# mmwrite(matrix_select_fn, csr_matrix(mtx_ipf[:,sel]))

# dbco = mtx_ipf[:,np.where(np.array(genes) == 'dbco')[0][0]]
# with open("dbco.txt", 'w') as f:
#   for v in dbco:
#     f.write(f'{v}\n')

In [ ]:
#@title entropy plot
fig, ax = plt.subplots(figsize=(5,5))


x = np.sort(adj_ent)[::-1]
y = np.arange(x.shape[0])

ax.scatter(x,y)
ax.set(**{
    "yscale": "symlog",
    "xscale": "symlog",
    "xlabel": "-log(entropy)",
    "ylabel": "Cell rank"
})

ind = np.where(x > cutoff)[0][-1]
ecutoff = x[ind]
ax.axvline(x=ecutoff, color="k")
ax.axhline(y=ind, color="k")


fig.show()

In [ ]:
df.to_csv('assignments.txt.gz', sep='\t', compression='gzip')